<a href="https://colab.research.google.com/github/ShainaBagri/ClassifyDocumentTopics/blob/main/proj1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget 'http://users.csc.calpoly.edu/~foaad/proj1F21_files.zip'
!unzip 'proj1F21_files.zip'

--2021-10-20 04:36:05--  http://users.csc.calpoly.edu/~foaad/proj1F21_files.zip
Resolving users.csc.calpoly.edu (users.csc.calpoly.edu)... 129.65.128.20
Connecting to users.csc.calpoly.edu (users.csc.calpoly.edu)|129.65.128.20|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 129006 (126K) [application/zip]
Saving to: ‘proj1F21_files.zip’

proj1F21_files.zip  100%[===================>] 125.98K   464KB/s    in 0.3s    

2021-10-20 04:36:06 (464 KB/s) - ‘proj1F21_files.zip’ saved [129006/129006]

Archive:  proj1F21_files.zip
   creating: proj1F21_files/
  inflating: proj1F21_files/proj1F21_1412_A.html  
  inflating: proj1F21_files/proj1F21_1412_B.html  
  inflating: proj1F21_files/proj1F21_1422_A.html  
  inflating: proj1F21_files/proj1F21_1422_B.html  
  inflating: proj1F21_files/proj1F21_1483_A.html  
  inflating: proj1F21_files/proj1F21_1483_B.html  
  inflating: proj1F21_files/proj1F21_1794_A.html  
  inflating: proj1F21_files/proj1F21_1794_B.html  
  inflating

In [ ]:
!pip install html2text

In [ ]:
import nltk
nltk.download('punkt')
# from nltk import PunktSentenceTokenizer
import pandas as pd
import numpy as np
import os
import random 
import html2text
from sklearn.feature_extraction.text import TfidfVectorizer
random.seed(123)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
sent_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
def tokenizeText(text):
  goodParas = []
  for para in text.split('\n\n\n'):
    sents = sent_tokenizer.tokenize(para.replace('\n', ' ').strip())
    goodSents = []
    for sent in sents:
      # TODO: lemmatize / convert contractions here?
      words = [word.lower() for word in nltk.word_tokenize(sent)]
      goodSents.append(words)
    
    goodParas.append(goodSents)
      
  return goodParas

def extractFeaturesDoc(tokText):
  def dummy(x):
    return x
  
  features = {}
  # tfidf
  tfidf = TfidfVectorizer(analyzer='word', tokenizer=dummy, preprocessor=dummy, token_pattern=None, max_df=0.75)
  features['tfidf'] = tfidf.fit_transform(tokText)

  return features

In [ ]:
dir = './proj1F21_files/'
# docs = pd.DataFrame(columns=['author', 'subject', 'rawText'])
docs = []
for document in os.listdir(dir):
  dict = {}
  docName = document.split("_")
  dict['author'] = docName[1]
  dict['subject'] = docName[2][0]
  with open(dir + document) as f:
    dict['htmlText'] = f.read()
    dict['rawText'] = html2text.html2text(dict['htmlText'])
    # tokenize 
    dict['tokenizedText'] = tokenizeText(dict['rawText']) 

  docs.append(dict)

# split into training and test sets
random.shuffle(docs)
split = int(len(docs) *0.8)
test_set = pd.DataFrame(docs[split:])
training_set = pd.DataFrame(docs[:split])





print(training_set['htmlText'][0])
print(training_set['rawText'][0])
print(training_set['tokenizedText'][0])





<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN"
        "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml">
<head>
  <meta http-equiv="Content-Type" content="text/html;charset=utf-8" />
</head>
<body>
<div style="width: 600px; margin: 20px auto; border: 1px solid #888; padding: 20px;">
  <p>One of my favorite people is my best friend Arya. She's one of those people that everybody gets along with. She's incredibly kind and patient with people. Her house was where we threw all the parties since her parents were very laidback. If I think back about high school, most of my best memories were at her house. We were the dynamic duo planners of our group. She was the eternal pacifist and is very welcoming. She'll always invite new people to hangouts and introduce to everybody and make sure they feel welcome.&nbsp;</p>
<p>&nbsp;</p>
<p>I think its important to have a friend who support you but also tells you when you're making